In [ ]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats.contingency import association
from scipy.stats import pearsonr

# Cleaning and graphs

In [ ]:
client_profiles=pd.read_csv("/Users/angie/Documents/Ironhack/ProjectTwo/Data/Original data/df_final_demo.txt")

In [ ]:
client_profiles.describe().round()

In [ ]:
#cleaning part
client_profiles.dropna(thresh=2, inplace=True, ignore_index=True)
client_profiles.clnt_age.fillna(client_profiles.clnt_age.mean(), inplace=True)
client_profiles['clnt_age']=client_profiles['clnt_age'].apply(int)

In [ ]:
client_profiles

In [ ]:
web_data_one=pd.read_csv("/Users/angie/Documents/Ironhack/ProjectTwo/Data/Original data/df_final_web_data_pt_1.txt")
web_data_two=pd.read_csv("/Users/angie/Documents/Ironhack/ProjectTwo/Data/Original data/df_final_web_data_pt_2.txt")

In [ ]:
web_data_two.isna().sum()

In [ ]:
#cleaning part
client_interactions=pd.concat([web_data_one,web_data_two], ignore_index=True)
client_interactions.drop_duplicates(inplace=True, ignore_index=True)
client_interactions['date_time']=pd.to_datetime(client_interactions.date_time,format='%Y-%m-%d %H:%M:%S')

In [ ]:
client_interactions

In [ ]:
client_experiment=pd.read_csv("/Users/angie/Documents/Ironhack/ProjectTwo/Data/Original data/df_final_experiment_clients.txt")

In [ ]:
client_experiment.isna().sum()

In [ ]:
#cleaning part
client_experiment.dropna(inplace=True, ignore_index=True)

In [ ]:
client_experiment

### Who are the primary clients using this online process?
between 24 and 63 years. Having to mayor groups. One between 25 and 32 years and the other between 49 and 63 years

### Are the primary clients younger or older, new or long-standing?
They are mix. The clients that have been at least 5 years (that is the hihger amount of customers) with the company are youngers and the ones that have been more than 12 years are more seniors.

In [ ]:
client_profiles.describe()

In [ ]:
#crosstable: count for clients ages in order to check the primary clients
age_count=pd.crosstab(index=client_profiles.clnt_age, columns='Count')
age_count_sorted=age_count.sort_values(by='Count', ascending=False)
age_count_sorted=age_count_sorted.head(30)

In [ ]:
plt.figure(figsize=(20, 7))
sns.countplot(data=client_profiles, x='clnt_age', palette="Set3", hue='clnt_age', legend=False)

In [ ]:
#plot zooming the higher amount of clients
plt.figure(figsize=(25, 7))
sns.barplot(x=age_count_sorted.index, y=age_count_sorted["Count"], palette="Set3",hue=age_count_sorted.index, legend=False)

In [ ]:
#Data base filtering the customer between the ages of 24 and 63 years old.
#following by plots in order to analyze the "ternure age" and the "age of the customers"
client_profiles_filter=client_profiles
client_profiles_filter["filter_age"]=client_profiles_filter["clnt_age"].apply(lambda value: value if value in age_count_sorted.index else "none")
filtered=client_profiles_filter["filter_age"]!="none"
client_profiles_filter[filtered]

In [ ]:

plt.figure(figsize=(50, 7))
sns.countplot(data=client_profiles_filter[filtered], x='clnt_tenure_yr', palette="Set3", hue="clnt_tenure_yr", legend=False)

In [ ]:
plt.figure(figsize=(50, 7))
sns.countplot(data=client_profiles_filter[filtered], x='clnt_tenure_yr', palette="Set3", hue="clnt_age", legend=False)

In [ ]:
plt.figure(figsize=(20, 7))
sns.countplot(data=client_profiles_filter[filtered][client_profiles_filter["clnt_tenure_yr"]==5], x='clnt_tenure_yr', palette="Set3", hue="clnt_age", legend=True)

In [ ]:
plt.figure(figsize=(20, 7))
sns.countplot(data=client_profiles_filter[filtered][client_profiles_filter["clnt_tenure_yr"]==6], x='clnt_tenure_yr', palette="Set3", hue="clnt_age", legend=True)

In [ ]:
#checking the amount of years that a customer between 49 and 63 years old have been with the company
plt.figure(figsize=(20, 7))
sns.countplot(data=client_profiles_filter[filtered][client_profiles_filter["clnt_age"].between(49,63)], x='clnt_tenure_yr', palette="Set3", hue="clnt_age", legend=True)

In [ ]:
#checking the amount of years that a customer between 25 and 32 years old have been with the company
plt.figure(figsize=(20, 7))
sns.countplot(data=client_profiles_filter[filtered][client_profiles_filter["clnt_age"].between(25,32)], x='clnt_tenure_yr', palette="Set3", hue="clnt_age", legend=True)

# Performance Metrics

In [ ]:
#creating a list of clien_id that reach "confirm" step
confirm_df=client_interactions[client_interactions.process_step=="confirm"]
client_id=list(confirm_df.client_id.unique())
filtered_id=client_interactions[client_interactions["client_id"].isin(client_id)]
filtered_id.sort_values(by=["client_id","date_time"], inplace=True)
#The one who do not reach "confirm"
filtered_no_confirm=client_interactions[~client_interactions["client_id"].isin(client_id)]

In [ ]:
id_didnot_completed=client_interactions[~client_interactions["client_id"].isin(client_id)]
no_complete_client_id=list(id_didnot_completed.client_id.unique())

In [ ]:
#creating a list of the customer whom complete the whole steps until reach "confirm" and the customers whom reach
#"confirm but come back to last step instead of moving forward"
steps=["start","step_1","step_2","step_3","confirm"]
finals_ids=[]
step_1=[]
step_2=[]
step_3=[]
confirm=[]
fail_steps=[]
fail_ids=[]

for item in client_id:
    df=filtered_id[filtered_id.client_id==item]
    if df.client_id.count()==5:
        psteps=list(df.process_step)
        if psteps==steps:
            finals_ids.append(item)
            time=list(df.date_time)
            step_1.append((time[1]-time[0]).total_seconds())
            step_2.append((time[2]-time[1]).total_seconds())
            step_3.append((time[3]-time[2]).total_seconds())
            confirm.append((time[4]-time[3]).total_seconds())
        else:
            fail_ids.append(item)
            fail_steps.append(list(df.process_step))
    else:
        fail_ids.append(item)
        fail_steps.append(list(df.process_step))


In [ ]:
#DataFrame of clients whom succeed
clients_confirm=pd.DataFrame({"client_id": finals_ids, "time_step_1": step_1, "time_step_2": step_2, "time_step_3": step_3,"time_confirm": confirm})



In [ ]:
#Creating DataFrame for the ones that who did not succeed in reach to "confirm" step:
not_complete=[]
not_complete_steps=[]

for item in no_complete_client_id:
    df=client_interactions[client_interactions.client_id==item]
    not_complete.append(item)
    not_complete_steps.append(list(df.process_step))


mode_not_complete=[]
for results in not_complete_steps:    
    for item in results:
        frequency_dict={}
        if item in frequency_dict:
            frequency_dict[item]+=1
        else:
            frequency_dict[item]=1
    maxi=max(frequency_dict.values())
    for key,value in frequency_dict.items():
        if value==maxi:
            mode_not_complete.append(key)

In [ ]:
total_not_complete=pd.DataFrame({"client_id":not_complete, "steps": not_complete_steps,"mode": mode_not_complete})
total_not_complete["result"]="not succeed"
total_not_complete["Time_AVG"]=0
total_not_complete

In [ ]:
# Total count unique client_id that do not interact but reach "confirm" - amt client error:
no_confirm_df=client_interactions[client_interactions.process_step!="confirm"]
interaction_cid=list(no_confirm_df.client_id.unique())
client_not_reach_cofirmation=set(client_id)-set(interaction_cid)


#Creating a dataframe with the amount of clients who do and do not succeed:

proportions=pd.DataFrame([int(client_interactions.client_id.nunique()), int(filtered_no_confirm.client_id.nunique()),int(filtered_id.client_id.nunique()), int(len(fail_ids)),  int(clients_confirm.client_id.nunique()), int(len(client_not_reach_cofirmation))], index=[
    "total_amt_clients",
    "amt_clients_not_reach_confirm",
    "amt_clients_reach_confirm",
    "amt_clients_reach_confirm_but_when_back",
    "amt_clients_reach_confirm_straightforward","amt_client_error"])

proportions.columns=["amount"]

proportions

In [ ]:
client_profiles[client_profiles["client_id"]==355337]

In [ ]:
client_experiment[client_experiment["client_id"]==355337]

In [ ]:
clients_confirm[clients_confirm["client_id"]==355337]

### Total unique clients_proflie: 70595
### Total unique clients_experiment_table: 50500
### Total unique clients in experiment_table but they do not have a profile: 12
### Total unique clients that succeed but they do not have a profile: 15809
### Total unique clients that succeed but they are not Test neither Control: 22137

In [ ]:
total_clients_profile=list(client_profiles.client_id.unique())
total_clients_experiment=list(client_experiment.client_id.unique())
clients_no_info_profile=[item for item in total_clients_experiment if item not in total_clients_profile]
id_clients_suceed=list(clients_confirm.client_id.unique())
clients_id_info_profile_succeed=[item for item in id_clients_suceed if item not in total_clients_profile]


In [ ]:
##Table with the clients who reach the confirm step, but not straightforward
mode_error=[]
for results in fail_steps:    
    for item in results:
        frequency_dict={}
        if item in frequency_dict:
            frequency_dict[item]+=1
        else:
            frequency_dict[item]=1
    maxi=max(frequency_dict.values())
    for key,value in frequency_dict.items():
        if value==maxi:
            mode_error.append(key)
    
         

In [ ]:
clients_errors=pd.DataFrame({"client_id":fail_ids, "steps":fail_steps, "mode":mode_error})
clients_errors["result"]="error"
clients_errors["Time_AVG"]=0
clients_errors

In [ ]:
#mergin client_experiment to clients_confirm table in order to check whom were and were not involved in the test
variation_client=pd.merge(clients_confirm, client_experiment, on='client_id', how='left')
variation_client.isna().sum()

In [ ]:
succeed_client=variation_client.dropna()

In [ ]:
succeed_client["total_time"]=(succeed_client["time_step_1"]+succeed_client["time_step_2"]+succeed_client["time_step_3"]+succeed_client["time_confirm"]).round(0)

In [ ]:
#Cheking out the ones that take more than one day in reach "confirm", that are affecting the AVG of the results:
succeed_client[succeed_client["total_time"]>=86400]

In [ ]:
#Takign out the ones that take more than one day in reach "confirm":
experiment_client_less_one_day=succeed_client[succeed_client["total_time"]<=86400]

In [ ]:
experiment_client_less_one_day["Time_AVG"]=(experiment_client_less_one_day.total_time/4).round(1)
experiment_client_less_one_day["result"]="succeed"

In [ ]:
#example of special client that took two days from start until step_1
client_interactions[client_interactions["client_id"]==324094]

In [ ]:
#Check if all client_id in experiment, are in the client_interaction: must be 0 the return
total_client_interactions=list(client_interactions.client_id.unique())
completation_yes=client_experiment[client_experiment["client_id"].isin(total_client_interactions)]
len(client_experiment.client_id)-len(completation_yes)

In [ ]:
#mergin a table by id, variation, and result test:
merge_results=pd.concat([clients_errors,experiment_client_less_one_day,total_not_complete], axis=0)
finals_error_and_succeed=merge_results.drop(["steps", "time_step_1",	"time_step_2",	"time_step_3",	"time_confirm","Variation"], axis=1)
finals_error_and_succeed

In [ ]:
final_experiment_result=pd.merge(client_experiment, finals_error_and_succeed, how="left", on="client_id")
final_experiment_result=final_experiment_result.dropna(subset="result")

In [ ]:
final_experiment_result

# Hypothesis Testing

# Completion Rate:
Hypothesis:


 - H0: The difference in completion rate, from Test and Control group is not statistically significant  $cr_{control} = cr_{test}$
 - H1: The difference completion rate, from Test and Control group is statistically significant          $cr_{control} \ne cr_{test}$

        -Result: ztest_statistic: -1.4948643657156728, ztest:_p_value: 0.13494984383765263

The null hypothesis is not rejected, being the P_value higher than 0.05. Which leads to a none significant statistically difference




In [ ]:
size_sample_test=client_experiment[client_experiment["Variation"]=="Test"]["Variation"].count()
size_sample_control=client_experiment[client_experiment["Variation"]=="Control"]["Variation"].count()

In [ ]:
size_success_test=experiment_client_less_one_day[experiment_client_less_one_day["Variation"]=="Test"]["Variation"].count()
size_success_control=experiment_client_less_one_day[experiment_client_less_one_day["Variation"]=="Control"]["Variation"].count()

In [ ]:
z_stat, p_value = proportions_ztest([size_success_test,size_success_control],[size_sample_test,size_sample_control])
z_stat, p_value

## Completion Rate with a Cost-Effectiveness Threshold

Hypothesis:

 - H0: The difference does not exceed the 5% theshold             H0 < H1
 - H1: The difference does exceed the 5% theshold                 H0 > H1

        -Result: ztest_statistic: -1.4948643657156728, ztest:_p_value: 0.9325250780811737

The null hypothesis is rejected, being the P_value higher than 0.05. Which does exceed the 5% theshold

In [ ]:
z_stat_larger, p_value_larger = proportions_ztest([size_success_test,size_success_control],[size_sample_test,size_sample_control], value=0.05, alternative='larger')
z_stat_larger, p_value_larger

## Relation between the control and test groups agains the results

$H_{0}:$ both columns are independent (not related)

$H_{1}:$ both columns are related

    -Result: Pvalue: 1.988446134802158e-26, less than 0, therefore we reject the HO hypothesis. And both columns are related
            With the Cramer's V I can deduct that even there is a dependenci between this variables, the strength is weak. Being the  association: 0.04841667649303389, lees than 1

In [ ]:
crosstab_result = pd.crosstab(final_experiment_result['result'], final_experiment_result['Variation'], normalize=False)
crosstab_result

In [ ]:
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result)

chi2_statistic, chi2_p_value

In [ ]:
association(crosstab_result, method="cramer")

## Age hypothesis

    - H0 : Results are independent of the age
    - H1 : Results are dependent of the age

### Results:
    - from the chi2 test the Pvalue is: 3.379010337241941e-85. Less than 0.05. Therefore we reject the null hypothesis. Therefore the results and the age have a significant association
    - About the stregth of this association, throught the Cramer's V result: 0.06956040694531841, close to zero. I can deduct that the asocciation tends to be weak

In [ ]:
filtering_age_ternure_yr_gender=client_profiles.drop(["clnt_tenure_mnth","num_accts","bal","calls_6_mnth","logons_6_mnth"], axis=1)
profiles_experiment_result=pd.merge(final_experiment_result, filtering_age_ternure_yr_gender, how="left", on="client_id")
profiles_experiment_result

In [ ]:
crosstab_result_age=pd.crosstab([profiles_experiment_result['Variation'],profiles_experiment_result['result']],profiles_experiment_result['clnt_age'], normalize=False)
crosstab_result_age

In [ ]:
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_age)

chi2_statistic, chi2_p_value

In [ ]:
association(crosstab_result_age, method="cramer")

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(crosstab_result_age, annot=False, cmap='coolwarm')
plt.xticks(rotation=60)


## Ternure hypothesis

    - H0 : Results are independent of the ternure yrs
    - H1 : Results are dependent of the ternuere yrs
Results:
- from the chi2 test the Pvalue is: 5.764857428635835e-09. Less than 0.05. Therefore we reject the null hypothesis. Therefore the results and the ternure yrs have a significant association
- About the stregth of this association, throught the Cramer's V result: 0.04039377130381852, close to zero. I can deduct that the asocciation tends to be weak

In [ ]:
crosstab_result_ternure=pd.crosstab([profiles_experiment_result['Variation'],profiles_experiment_result['result']],profiles_experiment_result['clnt_tenure_yr'], normalize=False)
crosstab_result_ternure

In [ ]:
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_ternure)

chi2_statistic, chi2_p_value

In [ ]:
association(crosstab_result_ternure, method="cramer")

In [ ]:
sns.heatmap(crosstab_result_ternure, annot=False, cmap="YlGnBu")

## Checking the Distribution of the test amount of parcipants by age and ternure

In [ ]:
#by age:
sns.kdeplot(data=profiles_experiment_result, fill=True 
, x='clnt_age', hue="Variation");

In [ ]:
# by ternure:
sns.kdeplot(data=profiles_experiment_result, fill=True 
, x='clnt_tenure_yr', hue="Variation");

## Checkign the results with customer than have been more than 20yrs

In [ ]:
checking_ternure_than20=profiles_experiment_result[profiles_experiment_result["clnt_tenure_yr"]>=20]
checking_ternure_than20

In [ ]:
cross_ternure_than20=pd.crosstab([checking_ternure_than20["Variation"],checking_ternure_than20["result"]],checking_ternure_than20["clnt_age"])

In [ ]:
sns.heatmap(cross_ternure_than20, annot=False, cmap="YlGnBu")

## Correlation coefficients

    - The varibles age and ternure yrs do not have a linear relationship between, being the Pearson result: 0.3111478676205681, close to zero rather than 1. Also for the result of the Spearman: 0.30277137656233305 I can mentioned that there even not a monotonic relationship between them.
    - Would be nice mentioned that the Pvalue from the Pearson test is lest than 0.05, therefore these two variables are not independent

In [ ]:
table_for_correlation=profiles_experiment_result[~profiles_experiment_result["clnt_age"].isna()]

In [ ]:
correlation_coef, p_value = pearsonr(table_for_correlation['clnt_age'], table_for_correlation['clnt_tenure_yr'])
correlation_coef, p_value

In [ ]:
table_for_correlation['clnt_age'].corr(table_for_correlation['clnt_tenure_yr'])

In [ ]:
table_for_correlation['clnt_age'].corr(table_for_correlation['clnt_tenure_yr'], method="spearman")

In [ ]:
sns.lmplot(data=table_for_correlation,
            x='clnt_age',
            y='clnt_tenure_yr',line_kws={"color": "yellow"});

In [ ]:
correlation_time_age=table_for_correlation[table_for_correlation["result"]=="succeed"]

In [ ]:
correlation_time_age.sort_values(by="Time_AVG")

In [ ]:
correlation_coef, p_value = pearsonr(correlation_time_age['clnt_age'], correlation_time_age['Time_AVG'])
correlation_coef, p_value

In [ ]:
sns.lmplot(data=correlation_time_age,
            x='clnt_age',
            y='Time_AVG',line_kws={"color": "yellow"});

 # Relation between the age and the time in minutes:

     - I can deduct that in base of the correlation result from the Pearson test, there is not a linear relationship between the mentioned variables. Therefore the age did not afect that the AVG in minutes for the new update really change.
     - With the spearman method : 0.26684562569386716, close to zero, we can also indicate that is a wak or no monotonic relationship
     - Though the Pvalue is less than 0.05 : 1.1697643971178092e-136, therefore there is a correlation but not linear

In [ ]:
correlation_time_age_drop=correlation_time_age.drop([33038,19986])

In [ ]:
correlation_time_age_drop["AVG_minutes"]=correlation_time_age_drop["Time_AVG"].apply(lambda x: x/60).round(2)

In [ ]:
correlation_time_age_drop

In [ ]:
correlation_coef, p_value = pearsonr(correlation_time_age_drop['clnt_age'], correlation_time_age_drop['AVG_minutes'])
correlation_coef, p_value

In [ ]:
sns.lmplot(data=correlation_time_age_drop,
            x='clnt_age',
            y='AVG_minutes',hue="Variation");

In [ ]:
correlation_time_age_drop['clnt_age'].corr(correlation_time_age_drop['AVG_minutes'], method="spearman")

### Add the AVG in minutes to the df of the experiment_client

In [ ]:
experiment_client_less_one_day["AVG_minutes"]=experiment_client_less_one_day["Time_AVG"].apply(lambda x: x/60).round(2)
experiment_client_less_one_day

In [ ]:
outliers=succeed_client[succeed_client["total_time"]>=86400]

In [ ]:
succeed_client

In [ ]:
outliers

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
sns.boxplot(data=succeed_client, y="total_time", hue="Variation", ax=ax)
plt.show()
